En este bloque de código usted va a hallar:

- Instalación de las librerías necesarias
- Verificación de instalación de algunas librerías esenciales
- Verificación de la correcta importación de TensorFlow Serving API
- Importación de librerías esenciales para la manipulación de datos y ML

In [9]:

!pip install tensorflow==2.10.0
!pip install tensorflow-serving-api==2.10.0
!pip install protobuf==3.19.6
!pip install apache-beam==2.40.0
!pip install google-api-python-client==2.70.0
!pip install google-cloud-bigquery==2.34.4
!pip install tfx==1.10.0
!pip install ml-metadata==1.11.0
!pip install tensorflow-data-validation==1.10.0
!pip install tensorflow-transform==1.10.0
!pip install pandas scikit-learn requests


libres = ["pandas", "tensorflow", "sklearn", "tfx", "ml_metadata", "requests"]
for lib in libres:
    try:
        exec(f"import {lib}")
        print(f"{lib} ya está instalado, siga")
    except ImportError:
        print(f"{lib} no está instalado...aun")
        !pip install {lib} --quiet

try:
    from tensorflow_serving.apis import classification_pb2
    print("¡Importación de tensorflow_serving exitosa!")
except ImportError as e:
    print(f"Error de importación: {e}")
    
import pandas as pd
import os
from sklearn.feature_selection import SelectKBest, f_classif
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

  Using cached tensorflow-2.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9

ModuleNotFoundError: No module named 'tensorflow_model_analysis'

En este segmento de código usted va a hallar:

- Carga de datos desde un archivo CSV
- Separación de variables predictoras y la variable objetivo

In [ ]:
archivo_csv = "covertype_training.csv"
try:
    df = pd.read_csv(archivo_csv)
    print('Lo hemos leido bien')
except Exception as e:
    print(f"⚠️ Error al leer el archivo: {e}")
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']

En este segmento de código usted va a hallar:

- Selección de las 10 mejores características usando ANOVA F-test
- Obtener los nombres de las columnas seleccionadas

In [ ]:
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Características seleccionadas:", selected_features)

En este segmento de código usted va a hallar:

- Creación de un pipeline de TFX para procesamiento de datos
- Configurar el contexto interactivo de TFX
- Ingesta de datos con ExampleGen
- Generación de estadísticas con StatisticsGen
- Inferencia de esquema de datos con SchemaGen
- Validación de los datos con ExampleValidator

In [ ]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform
context = tf.dsl.Context()
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

En este segmento de código usted va a hallar:
- Definir una función de preprocesamiento de datos con TensorFlow Transform
- Aplicar la transformación a los datos con Transform

In [ ]:
from tfx.components import Transform
def preprocessing_fn(inputs):
    import tensorflow_transform as tft
    outputs = {}
    for key in inputs:
        outputs[key] = tft.scale_to_0_1(inputs[key])
    return outputs
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn=preprocessing_fn)
context.run(transform)

En este segmento de código usted va a hallar:

- Configuración del almacén de metadatos
- Obtener y mostrar los tipos de artefactos almacenados
- Obtener y mostrar todos los artefactos registrados

In [ ]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = './metadata.db'
store = metadata_store.MetadataStore(connection_config)
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type)
artifacts = store.get_artifacts()
for artifact in artifacts:
    print(artifact)

In [ ]:
from tfx.orchestration.experimental.interactive import visualizations

# Visualizar anomalías
visualizations.display_anomalies(example_validator.outputs['anomalies'])

En este segmento de código usted va a hallar:

- Guardar el esquema actualizado en un archivo
- Importar el esquema actualizado al pipeline


In [ ]:
from tfx.utils import io_utils
from tfx.components import ImportSchemaGen
schema_path = './schema.pbtxt'
io_utils.write_pbtxt_file(schema_path, schema_gen.outputs['schema'].get())
import_schema_gen = ImportSchemaGen(schema_file=schema_path)
context.run(import_schema_gen)
statistics_gen_updated = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=import_schema_gen.outputs['schema'])
context.run(statistics_gen_updated)

En este segmento de código usted va a hallar:
- Validar datos con el esquema actualizado
- Función para rastrear artefactos principales en el almacén de metadatos

In [ ]:
example_validator_updated = ExampleValidator(
    statistics=statistics_gen_updated.outputs['statistics'],
    schema=import_schema_gen.outputs['schema'])
context.run(example_validator_updated)
def get_parent_artifacts(store, artifact_id):
    events = store.get_events_by_artifact_ids([artifact_id])
    execution_ids = [event.execution_id for event in events]
    executions = store.get_executions_by_id(execution_ids)
    parent_artifacts = []
    for execution in executions:
        parent_events = store.get_events_by_execution_ids([execution.id])
        parent_artifacts.extend([event.artifact_id for event in parent_events])
    return parent_artifacts

# Ejemplo de uso
artifact_id = 1  
parent_artifacts = get_parent_artifacts(store, artifact_id)
print("Artefactos principales:", parent_artifacts)